In [47]:
import numpy as np
import pandas as pd
import scipy.stats
import pymc3 as pm
import arviz as az

In [78]:
class Leitner:
    def __init__(self, n_item, delay_factor, delay_min):

        box = np.full(n_item, -1)
        due = np.full(n_item, -1)

        self.n_item = n_item

        self.delay_factor = delay_factor
        self.delay_min = delay_min

        self.box = box
        self.due = due

    def update_box_and_due_time(self, last_idx, last_was_success, last_time_reply):

        if last_was_success:
            self.box[last_idx] += 1
        else:
            self.box[last_idx] = max(0, self.box[last_idx] - 1)

        delay = self.delay_factor ** self.box[last_idx]
        # Delay is 1, 2, 4, 8, 16, 32, 64, 128, 256, 512 ... minutes
        self.due[last_idx] = last_time_reply + self.delay_min * delay

    def _pickup_item(self, now):

        seen = np.argwhere(np.asarray(self.box) >= 0).flatten()
        n_seen = len(seen)

        if n_seen == self.n_item:
            return np.argmin(self.due)

        else:
            seen__due = np.asarray(self.due)[seen]
            seen__is_due = np.asarray(seen__due) <= now
            if np.sum(seen__is_due):
                seen_and_is_due__due = seen__due[seen__is_due]

                return seen[seen__is_due][np.argmin(seen_and_is_due__due)]
            else:
                return self._pickup_new()

    def _pickup_new(self):
        return np.argmin(self.box)

    def ask(self, now, last_was_success, last_time_reply, idx_last_q):

        if idx_last_q is None:
            item_idx = self._pickup_new()

        else:

            self.update_box_and_due_time(
                last_idx=idx_last_q,
                last_was_success=last_was_success,
                last_time_reply=last_time_reply,
            )
            item_idx = self._pickup_item(now)

        return item_idx

In [79]:
np.random.seed(123)
n_item = 500

ss_n_iter = 100
time_per_iter = 4
n_sec_day = 24 * 60 ** 2
n_ss = 100
eval_ts = n_ss * n_sec_day
review_ts = np.hstack([
        np.arange(x, x + (ss_n_iter * time_per_iter), time_per_iter)
        for x in np.arange(0, n_sec_day * n_ss, n_sec_day)
    ])

# thr = 0.90

In [80]:
def run(param, agent=0, c=1e-05):
    bkp = []

    n_total_iter = len(review_ts)

    rd = np.random.random(size=n_total_iter)

    is_it_spec = len(np.asarray(param).shape) > 1

    lei = Leitner(n_item=n_item, delay_min=4, delay_factor=2)

    n_pres = np.zeros(n_item, dtype=int)
    last_pres = np.zeros(n_item)

    last_item = None
    last_success = None
    last_ts = None

    for i, ts in enumerate(review_ts):

        item = lei.ask(
            now=ts,
            idx_last_q=last_item,
            last_was_success=last_success,
            last_time_reply=last_ts)

        if is_it_spec:
            α, β = param[item, 0], param[item, 1]
        else:
            α, β = param[0], param[1]

        if not n_pres[item]:
            p = 0
        else:
            lp = - α * (1 - β) ** (n_pres[item] - 1) * (ts - last_pres[item]*c)
            p = np.exp(lp)

        success = p > rd[i]

        # Backup
        if n_pres[item] > 0:
            bkp.append({
                "agent": agent,
                "item": item, "success": success, 
                "n_rep": n_pres[item]-1, 
                "delta_rep": ts - last_pres[item]})

        # Update values
        n_pres[item] += 1
        last_pres[item] = ts
        last_success = success
        last_ts = ts
        last_item = item
    return pd.DataFrame(bkp)

In [36]:
# param = [0.0006, 0.44] # [[0.0006, 0.44] for _ in range(n_item)]
# param = np.asarray(param)

In [37]:
df = run([0.0006, 0.44])
df

,agent,item,success,n_rep,delta_rep
0,0,0,True,0,4.0
1,0,0,True,1,8.0
2,0,1,True,0,8.0
3,0,1,True,1,8.0
4,0,2,True,0,8.0
...,...,...,...,...,...
542,0,33,True,10,268.0
543,0,48,True,4,76.0
544,0,20,True,12,264.0
545,0,51,True,3,44.0


In [38]:
df.success.mean()

0.8592321755027422

In [63]:
0.025*1e5

2500.0

In [64]:
def truncnorm(mu, sigma, lower, upper, size):
    
    # a, b = (lower - mu) / sigma, (upper - mu) / sigma
    return scipy.stats.truncnorm.rvs(a=lower, b=upper, loc=mu, scale=sigma, size=size)

In [140]:
n = 20
param = np.vstack((truncnorm(0.5, 1.0, lower=0.02, upper=2500.0, size=n), 
                   truncnorm(0.5, 0.2, lower=0, upper=1, size=n))).T
pd.DataFrame(param, columns=('α', 'β'))

,α,β
0,1.834395,0.604841
1,1.522848,0.594767
2,0.562377,0.647283
3,0.575712,0.567568
4,1.284941,0.540603
5,0.909137,0.692725
6,1.343842,0.590657
7,1.579216,0.502992
8,1.791002,0.533429
9,1.609947,0.613586


In [141]:
r = []
for i, pr in enumerate(param):
    r.append(run(pr, agent=i))

df = pd.concat(r)
df

,agent,item,success,n_rep,delta_rep
0,0,0,False,0,4.0
1,0,0,False,1,4.0
2,0,0,False,2,4.0
3,0,0,False,3,4.0
4,0,0,False,4,4.0
...,...,...,...,...,...
9725,19,269,False,15,4.0
9726,19,269,False,16,4.0
9727,19,269,False,17,4.0
9728,19,269,False,18,4.0


In [142]:
df.groupby("agent")["success"].mean()

agent
0     0.511819
1     0.508734
2     0.558727
3     0.508116
4     0.477284
5     0.581038
6     0.509298
7     0.449519
8     0.469352
9     0.521900
10    0.549902
11    0.501592
12    0.556300
13    0.501849
14    0.454424
15    0.480357
16    0.516908
17    0.574611
18    0.504366
19    0.516238
Name: success, dtype: float64

In [143]:
for i in range(n):
    
    df_a = df[df.agent == i]
    n_rep, delta_rep, success = df_a.n_rep.values, df_a.delta_rep.values, df_a.success.astype(bool).values
    eps = np.finfo(float).eps
    
    with pm.Model() as model:

        α = pm.TruncatedNormal('α', mu=2.0, sigma=1.0, lower=0.02, upper=2500.0)
        β = pm.TruncatedNormal('β', mu=0.1, sigma=0.1, lower=0.0, upper=1.0)

        lp = - delta_rep * α * (1 - β) ** n_rep * 1e-05
        logit_p = lp - np.log(1-np.exp(lp) + eps)

        recall = pm.Bernoulli('recall', logit_p=logit_p, observed=success)

    with model:
        trace = pm.sample(1000, tune=1000, chains=2, return_inferencedata=True)
    trace_sm = az.summary(trace)
    print(trace_sm.loc['α', 'mean'], param[i, 0])
    print(trace_sm.loc['β', 'mean'], param[i, 1])

/Users/aureliennioche/.pyenv/versions/3.9.1/lib/python3.9/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log1p
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
Auto-assigning NUTS sampler...
INFO:pymc3:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
INFO:pymc3:Multiprocess sampling (2 chains in 2 jobs)
NUTS: [β, α]
INFO:pymc3:NUTS: [β, α]


KeyboardInterrupt: 

In [121]:
import theano.tensor as tt
import theano

In [134]:
eps = np.finfo(float).eps

In [147]:
with pm.Model() as model:
    
    mu_a = 2.0
    sigma_a = 2.0
    mu_b = 0.5
    sigma_b = 0.1
    # mu_a = pm.TruncatedNormal("mu_a", mu=2.0, sigma=1, lower=0.02, upper=2500.0)
    # sigma_a = pm.HalfNormal("sigma_a", 5.0)
    # mu_b = pm.TruncatedNormal("mu_b", mu=0.3, sigma=1, lower=0, upper=1)
    # sigma_b = pm.HalfNormal("sigma_b", 5.0)
    
    α = pm.TruncatedNormal('α', mu=mu_a, sigma=sigma_a, lower=0.02, upper=2500.0, shape=n)
    β = pm.TruncatedNormal('β', mu=mu_b, sigma=sigma_b, lower=0.0, upper=1.0, shape=n)
    
    # list_alpha = [α[i] for i in agent]
    # list_beta = [β[i] for i in agent]
    
    # alpha = tt.stack(list_alpha)
    # beta = tt.stack(list_beta)
    
    to_concat = []
    
    for i in range(n):
        df_a = df[df.agent == i]
        n_rep, delta_rep = df_a.n_rep.values, df_a.delta_rep.values
        
        alpha = α[i]
        beta = β[i]
        
        lp = - delta_rep * alpha * (1 - beta) ** n_rep * 1e-05
        logit_p = lp - np.log(1 - np.exp(lp) + eps)
        
        to_concat.append(logit_p)

#     lp = - delta_rep * alpha * (1 - beta) ** n_rep * 1e-05
#     logit_p = lp - np.log(1 - np.exp(lp) + eps)
    # logit_p = alpha / pm.math.maximum(alpha)
    
    
    logit_p_tamere = tt.concatenate(to_concat)
    success = df.success.astype(bool).values
    
    recall = pm.Bernoulli('recall', logit_p=logit_p_tamere, observed=success)

In [ ]:
with model:
    trace = pm.sample(1000, tune=1000, chains=2, return_inferencedata=True)

trace_sm = az.summary(trace)

/Users/aureliennioche/.pyenv/versions/3.9.1/lib/python3.9/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log1p
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
Auto-assigning NUTS sampler...
INFO:pymc3:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
/Users/aureliennioche/.pyenv/versions/3.9.1/lib/python3.9/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log1p
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/Users/aureliennioche/.pyenv/versions/3.9.1/lib/python3.9/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log1p
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/Users/aureliennioche/.pyenv/versions/3.9.1/lib/python3.9/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log1p
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
Multiproce